In [1]:
! nvidia-smi

Tue Oct 14 14:36:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
! pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 24.0 MB/s eta 0:00:00


#sentencepiece:
- This is a tokenizer, specifically a subword tokenizer. It's used to break down text into smaller units (subwords) which can be helpful for handling out-of-vocabulary words in natural language processing tasks.
sacrebleu: This library is used for evaluating the quality of machine translation. It provides a standardized way to calculate the BLEU score, which measures the similarity between a generated translation and a set of reference translations.
# rouge_score:
- This library is used for evaluating the quality of automatic summarization and machine translation. It calculates ROUGE (Recall-Oriented Understudy for Gisting Evaluation) scores, which measure the overlap of n-grams, word sequences, and pairs of words between a generated summary/translation and a reference.
# py7zr:
- This is a Python library for working with 7z archives (a type of compressed file). It allows you to extract files from or create 7z archives within your Python code. You likely installed this to handle the summarizer-data.zip file, which seems to be a 7z archive.

# sentencepiece (for Tokenization)

- Imagine you have the word "unbelievable". A word-based tokenizer would treat this as one token. However, if the model hasn't seen this word before, it won't know how to process it.

- A subword tokenizer like SentencePiece would break it down into smaller, known units, like "un", "believe", and "able". The model is more likely to have encountered these subwords, allowing it to understand the meaning of "unbelievable" even if it's a new word.

- Example: This is a conceptual example, actual implementation requires training a SentencePiece model.
- import sentencepiece as spm

- Let's assume you have a trained sentencepiece model
- sp = spm.SentencePieceProcessor()
-  sp.load('your_model.model')

- Example of encoding and decoding
- text = "This is an unbelievable example."
- encoded_text = sp.- encode_as_pieces(text)
- print(encoded_text) Output might look like: [' This', ' is', ' an', ' un', 'believe', 'able', '.', ]

- decoded_text = sp.decode_pieces(encoded_text)
- print(decoded_text) # Output: - This is an unbelievable example.

### Sacrebleu (for Machine Translation Evaluation)

- Suppose you have a reference translation: "The cat sat on the mat."

- And a generated translation: "A cat was sitting on the mat."

- sacrebleu would compare these two sentences and calculate a BLEU score based on the overlap of n-grams (sequences of words). A higher BLEU score indicates a better translation.

- Example:

- from sacrebleu.metrics import BLEU

- bleu = BLEU()
- references = ["The cat sat on the mat."]
- candidates = ["A cat was sitting on the mat."]

- score = bleu.corpus_score(candidates, [references])
- print(score.score) # This will output the BLEU score
### rouge_score (for Summarization Evaluation)

- Let's say your reference summary is: "The quick brown fox jumps over the lazy dog."

- And your generated summary is: "A quick fox jumps over a lazy dog."

- rouge_score would calculate ROUGE scores (like ROUGE-1, ROUGE-2, ROUGE-L) to see how much overlap there is between the words and phrases in the two summaries.

- Example:

- from rouge_score import rouge_scorer

- scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog.','A quick fox jumps over a lazy dog.')

- print(scores)
- Output will show precision, recall, and f-measure for ROUGE-1 and ROUGE-L


In [3]:
! pip install  accelerate -q


In [4]:
! pip install transformers

# Purpose of accelerate:
- Ease of Multi-Device Training : whether you are using multiple GPUs or TPUs, accelerate makes it easier to scale your model across devices with minimal code changes.
- Mixed Precision: It allows models to be training using mixed precision , which can speed up training and reduce memory usage.
- Zero Redundancy Optimizers(ZeRO): helps manage large models that may not fit entirely into the GPU memory , by allwoing part of model  to be offloaded to CPU or even SSD


#Mixed Precision:
- This refers to using a combination of different numerical precisions (like 16-bit and 32-bit floating-point numbers) during model training. Traditionally, models are trained using 32-bit precision (single-precision). However, using 16-bit precision (half-precision) for certain operations can significantly speed up training and reduce memory usage without losing much accuracy. accelerate helps manage which operations use which precision, making it easier to implement mixed precision training.
Ease of Multi-Device Training: This means that accelerate simplifies the process of training your machine learning models across multiple GPUs or other hardware accelerators like TPUs. Instead of writing complex code to distribute your model and data manually, accelerate provides tools to handle this automatically, allowing you to scale your training more easily.

# Zero Redundancy Optimizers (ZeRO):
- This is a set of techniques designed to train very large models that might not fit into the memory of a single GPU. ZeRO optimizers work by partitioning the model's state (like parameters, gradients, and optimizer states) across different devices (GPUs, CPUs, or even SSDs). This reduces the memory footprint on each individual device, enabling you to train models that are much larger than what would normally fit in memory.
In essence, accelerate is a library that helps you make your deep learning training more efficient, especially when working with large models or multiple devices.




In [5]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
from tqdm import tqdm
import torch


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration

In [7]:
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
ARTICLE_TO_SUMMARIZE = ("PG&E stated it scheduled the blackouts in response to forecasts for high winds "
                "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
                "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow.")

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt")

#Generate Summary
Summary_ids = model.generate(inputs["input_ids"])
print(tokenizer.batch_decode(Summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

California's largest electricity provider has turned off power to hundreds of thousands of customers.



# Tokenize Input:
- inputs = tokenizer (ARTICLE_TO_SUMMARIZE, return_tensors="pt"): This line tokenizes the

 ### ARTICLE_TO_SUMMARIZE.
 - return_tensors="pt" specifies that the output should be PyTorch tensors.
### Generate Summary:
- Summary_ids = model.generate(inputs["input_ids"]):
- This is the core of the summarization. The model.generate() method takes the tokenized input IDs and uses the loaded Pegasus model to generate the token IDs for the summary.
Decode and Print Summary:
- print(tokenizer.batch_decode(Summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]):
- This line converts the generated summary token IDs back into human-readable text.
### tokenizer.batch_decode(): -  

- Decodes a batch of token IDs.
skip_special_tokens=True: Removes special tokens (like padding or start/end tokens) from the output.

###clean_up_tokenization_spaces=False:
- Prevents the decoder from adding extra spaces around punctuation.
[0]: Selects the first (and in this case, only) summary in the batch.

Input IDs Explained

When you work with natural language processing models like Pegasus, they don't directly process raw text. Instead, they work with numerical representations of the text. This is where the tokenizer comes in.

Tokenization: The tokenizer takes your input text (ARTICLE_TO_SUMMARIZE) and breaks it down into smaller units called "tokens." These tokens can be words, sub-words, or even characters, depending on the tokenizer's design.
Mapping to IDs: Each unique token in the tokenizer's vocabulary is assigned a unique numerical ID. The tokenizer then replaces each token in your input text with its corresponding ID.
Input IDs: The result of this process is a sequence of numerical IDs, which are the "input IDs" you see in the code (inputs["input_ids"]). This sequence of numbers is what the model actually receives as input.
Think of it like converting words into a secret code that the model understands.

Flow of the Code

Now, let's look at the flow of the code cell (SssAjZpLmiyA) step by step:

Load Resources: The first two lines load the pre-trained PegasusForConditionalGeneration model and its associated AutoTokenizer from the specified path ("google/pegasus-xsum"). These are the tools needed for summarization.
Prepare Input Text: The ARTICLE_TO_SUMMARIZE variable holds the text you want to summarize.
Tokenize Input Text: The tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt") line takes the input text and converts it into the numerical input_ids that the model can process. The return_tensors="pt" ensures the output is in a PyTorch tensor format, which is compatible with the model.
Generate Summary IDs: The model.generate(inputs["input_ids"]) line is where the magic happens. You pass the input_ids to the loaded Pegasus model. The model then uses its training to generate a sequence of output IDs. These output IDs represent the tokens of the generated summary.
Decode Summary IDs: Finally, tokenizer.batch_decode(Summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] takes the generated Summary_ids and converts them back into human-readable text. It uses the tokenizer to map the IDs back to their corresponding tokens and then joins them to form the summary string. The options skip_special_tokens and clean_up_tokenization_spaces help to make the output clean and readable.
Print Summary: The print() function displays the generated summary.

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Fine tuning:

In [9]:
model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model) # load a tokenizer
model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [10]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2025-10-14 14:38:32--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2025-10-14 14:38:33--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.06s   

2025-10-14 14:38:33 (122 MB/s) - ‘summarizer-data.zip’ saved [7903594/7903

In [11]:
from datasets import load_from_disk

dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum # it converts arrow files to python dict

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])
#

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


### Preparing data for the Training for sequence to sequence Model

- We use attention mask, input_ids, labels for that ,
- we initially break the sequentce inot tokesn these token then converted into emebedings and attention mask further refien these embeddings to take care of multiple interpretation

# Tokenization:
- This is the very first step. As you mentioned, the raw text is broken down into smaller units called tokens. Tokens can be words, sub-words (like "run" and "##ning" for "running"), or even characters. The tokenizer uses a vocabulary to map these tokens to unique numerical IDs.
# Input IDs:
- These are the numerical representations of your tokens. After tokenization, each token in your input sequence is replaced by its corresponding ID from the tokenizer's vocabulary. This sequence of numerical IDs is what the model actually receives as input.
# Embeddings:
- While not explicitly mentioned in your bullet points, the model internally converts these input IDs into embeddings. Embeddings are dense vector representations of tokens that capture their semantic meaning and relationships with other tokens. Think of it as converting the numerical IDs into a rich, multi-dimensional space where similar words are closer together.
# Attention Mask:
- This is a crucial component for handling sequences of varying lengths and for models that use attention mechanisms (like the Transformer architecture, which Pegasus is based on).
When processing a batch of sequences, models often pad the shorter sequences with special "padding" tokens so that all sequences in the batch have the same length.
The attention mask is a binary mask (usually containing 0s and 1s) that tells the model which tokens are actual data and which are padding.
During the attention calculation, the model is instructed to ignore the padded tokens (by masking them out using the attention mask).
This is important because you don't want the model to pay attention to or learn from the padding tokens, as they don't contain any meaningful information from the original text.
So, the attention mask helps the model focus its "attention" only on the real tokens in the sequence. Your point about the attention mask "further refine these embeddings to take care of multiple interpretation" is partially correct in that attention mechanisms, guided by the mask, help the model weigh the importance of different parts of the input sequence when processing, which contributes to better understanding and interpretation.
# Labels:
- For training a sequence-to-sequence model, you also need labels. In a summarization task, the labels are the desired output summaries.
These summary texts also need to be tokenized and converted into input IDs, just like the input dialogues.
During training, the model learns to generate the sequence of summary token IDs based on the input dialogue token IDs.
So, to summarize the preparation process for a sequence-to-sequence summarization model:

Take the input dialogue and the target summary.
Tokenize both the dialogue and the summary to get sequences of input IDs.
Create an attention mask for the dialogue input IDs to indicate real tokens versus padding.
The tokenized summary IDs serve as the labels for training.
These processed inputs (input IDs, attention mask, and labels) are then fed into the model for training.

You are correct that you break the sequence into tokens and convert them into input IDs. The attention mask is then used to guide the model's attention mechanism, preventing it from focusing on padding tokens and helping it process sequences effectively.

In [13]:
#So we have to convert our seq to seq data in this format
def convert_examples_to_features(example_batch):
  input_embeddings = tokenizer( example_batch['dialogue'], max_length=1024, truncation=True)
  with tokenizer.as_target_tokenizer():
    target_embeddings = tokenizer(example_batch['summary'], max_length=128, truncation=True)
  return{
      "input_ids": input_embeddings["input_ids"],
      "attention_mask": input_embeddings["attention_mask"],
      "labels": target_embeddings["input_ids"]
  }

In [14]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [15]:
dataset_samsum_pt["train"] # dataset is updated

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

#Training Starts

In [16]:
# Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

 - DataCollactorforseq2seq is a special data collacctor designed for a seq-2-seq models that helps in preparing batches of data for training

In [17]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    eval_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [18]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

/tmp/ipython-input-3442823430.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


In [19]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: othakur (othakur-university-of-maryland) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
500,1.660400,1.482138


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=921, training_loss=1.8324690536878008, metrics={'train_runtime': 3573.0143, 'train_samples_per_second': 4.123, 'train_steps_per_second': 0.258, 'total_flos': 5531718781673472.0, 'train_loss': 1.8324690536878008, 'epoch': 1.0})

- ### num_train_epochs=1:
Explanation: An epoch represents one complete pass through the entire training dataset. If you have 1000 training examples and a batch size of 10, one epoch would involve 100 batches of data being fed into the model. Setting num_train_epochs to 1 means the model will see each training example exactly once during the training process.
Impact: The number of epochs is a crucial hyperparameter. More epochs can lead to better model performance as the model has more opportunities to learn from the data. However, too many epochs can lead to overfitting, where the model performs very well on the training data but poorly on unseen data. Setting it to 1 is often done for initial testing or when working with very large datasets where even one pass is computationally expensive.
- ### warmup_steps=500:
Explanation: This parameter is related to the learning rate scheduler. The learning rate determines how much the model's parameters are adjusted during each training step. A learning rate warm-up is a technique where the learning rate is gradually increased from a very small value to its initial value over a specified number of steps (in this case, 500 steps).
Impact: Warm-up helps to stabilize the training process at the beginning, especially when using large learning rates or optimizers like Adam. It prevents large gradient updates early in training that could potentially destabilize the model.
- ### Per_device_train_batch_size=1:
Explanation: This defines the number of training examples processed in a single forward and backward pass on each allocated device (GPU or CPU). So, if you have one GPU, the batch size is 1. If you had two GPUs, the total effective batch size without gradient accumulation would be 2.
Impact: Batch size affects the training stability and speed. Smaller batch sizes can introduce more noise into the gradient updates, which can sometimes help the model escape local minima but might also slow down convergence. Larger batch sizes can provide more stable gradient estimates and speed up training on powerful hardware, but they require more memory.
- ### per_device_eval_batch_size=1:
Explanation: Similar to per_device_train_batch_size, this sets the number of evaluation examples processed per device during evaluation.
Impact: The evaluation batch size primarily affects the speed of evaluation. It doesn't impact the model's learning process since evaluation doesn't involve gradient updates. You can often use a larger evaluation batch size than the training batch size if memory allows, as it can speed up the evaluation process.
- ### weight_decay=0.01:
Explanation: Weight decay is a regularization technique that adds a penalty to the loss function based on the magnitude of the model's weights. Specifically, it adds a term proportional to the sum of the squares of the weights (L2 regularization). The weight_decay value (0.01 in this case) is the coefficient for this penalty term.
Impact: Weight decay discourages the model from using very large weights, which can help prevent overfitting by making the model less sensitive to small changes in the input data. It encourages the model to find simpler solutions.
### logging_steps=10:
-Explanation: This parameter determines how frequently training progress is reported. Every logging_steps (10 in this case), the trainer will log information such as the current training loss, learning rate, and other relevant metrics.
Impact: Logging helps you monitor the training process and track the model's performance. By observing the logged metrics, you can identify issues like convergence problems or overfitting early on.
### eval_strategy='steps':
- Explanation: This specifies when evaluation should be performed during training. Setting it to 'steps' means evaluation will happen after a fixed number of training steps, as defined by eval_steps. Other options include 'epoch' (evaluate after each epoch) or 'no' (no evaluation during training).
Impact: The evaluation strategy allows you to monitor the model's performance on a separate validation set during training. This is crucial for detecting overfitting and for selecting the best model checkpoint.
### - eval_steps=500:
Explanation: When eval_strategy is set to 'steps', this parameter specifies the number of training steps after which an evaluation pass will be performed. In this case, evaluation will happen every 500 training steps.
Impact: Together with eval_strategy, this controls the frequency of evaluation. More frequent evaluation provides more detailed insights into the training progress but also adds overhead to the training time.
 ### save_steps=1e6:
- Explanation: This parameter determines how often model checkpoints are saved during training. A checkpoint is a snapshot of the model's parameters and optimizer state at a specific point in training. The value 1e6 represents 1,000,000 steps.
Impact: Saving checkpoints allows you to resume training from a specific point if it's interrupted or to use a checkpoint from an earlier stage of training if it performs better. A large save_steps value means checkpoints are saved very infrequently. This might be done to save disk space or if you only need the final model.
 ### gradient_accumulation_steps=16:
- Explanation: This technique is used to effectively increase the batch size without requiring more memory on each device. Instead of performing an optimization step after processing a single batch, the gradients are accumulated over multiple smaller batches (16 in this case). The optimization step is then performed using the accumulated gradients.
Impact: Gradient accumulation allows you to train with larger effective batch sizes than would otherwise fit into your device's memory. This can be beneficial for training very large models or when you have limited GPU memory. The effective batch size is per_device_train_batch_size * gradient_accumulation_steps. In your case, it's 1 * 16 = 16.

In [20]:
# Evaluation
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
    predictions, true_values = [], []
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                    attention_mask=inputs["attention_mask"].to(device),
                                    length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        predictions.extend(decoded_summaries)
        true_values.extend(target_batch)

    # finally compute and return rouge scores
    score = metric.compute(predictions=predictions, references=true_values)
    return score

In [21]:
! pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00


In [22]:
import evaluate
rouge_metric = evaluate.load('rouge')
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [23]:
score = calculate_metric_on_test_ds(dataset_samsum["test"][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text="dialogue", column_summary="summary")
#Directly use the scores without accessing fmeasure or mid
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
import pandas as pd
pd.DataFrame(rouge_dict, index=[f"pegasus"])

100%|██████████| 5/5 [00:10<00:00,  2.12s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.024481,0.0,0.024732,0.024589


In [24]:
score = calculate_metric_on_test_ds(dataset_samsum['test'], rouge_metric, trainer.model, tokenizer, batch_size=8, column_text="dialogue", column_summary="summary")

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f"pegasus"])

100%|██████████| 103/103 [12:42<00:00,  7.40s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018578,0.000331,0.018469,0.018481


# Interpreting Good Vs Bad Rouge Scores :
- this indicates a strong overlap between the generated summary and the reference summary, which is desirable in summarization tasks. FOr example, an F1 socre of 0.7 or higher across metrics is generally considered good.
# Scores between 0.5 and 0.7:
- Indicates moderate overlap.The summary might be capturing key points but is likely missing some structure or important information.
# Score Below 0.5
- Suggest a poor match between the generated and reference summaries. The model might be generating irrelevant or incomplete summaries that don't capture the ky ideas well.

# Entire Work flow:

Setup and Installations: You started by setting up your environment, installing necessary libraries like transformers, datasets, sacrebleu, rouge_score, py7zr, and accelerate. These libraries provide the tools needed for working with transformer models, datasets, and evaluation metrics. You also checked for GPU availability using nvidia-smi.
Loading the Model and Tokenizer: You loaded the pre-trained google/pegasus-cnn_dailymail model and its corresponding tokenizer. The tokenizer is essential for converting text data into a format that the model can understand (numerical IDs), and the pre-trained model provides a good starting point for the summarization task. You also moved the model to the available device (GPU or CPU).
Loading and Preparing the Dataset: You loaded the samsum_dataset from a zip file you downloaded. This dataset contains dialogue and summary pairs suitable for training a summarization model. You then explored the dataset structure, including the splits (train, validation, test), the number of examples in each split, and the features (dialogue and summary).
Tokenizing the Dataset: You defined a function convert_examples_to_features to tokenize the dialogue and summary pairs in the dataset. This function uses the loaded tokenizer to convert the text into input IDs and attention masks for the dialogues, and input IDs for the summaries (which serve as labels for training). You then applied this function to the entire dataset using the .map() method, creating the dataset_samsum_pt with the processed data.
Setting up for Training: You initialized a DataCollatorForSeq2Seq, which is used to prepare batches of data for training sequence-to-sequence models. This includes padding the sequences to a uniform length within each batch. You also defined TrainingArguments, which specify various hyperparameters and settings for the training process, such as the output directory, number of epochs, batch sizes, learning rate parameters, and evaluation strategy.
Initializing and Running the Trainer: You created a Trainer object, which orchestrates the training and evaluation loop. You provided the model, training arguments, tokenizer, data collator, and the train and validation datasets to the trainer. Finally, you called trainer.train() to start the fine-tuning process. The model trained for one epoch as specified in the TrainingArguments.
Evaluating the Model: You defined a function calculate_metric_on_test_ds to evaluate the model's performance on a given dataset split using a specified metric (ROUGE in this case). This function processes the data in batches, generates summaries using the trained model, decodes the generated summaries, and then computes the ROUGE scores by comparing the generated summaries to the reference summaries.
Loading the Evaluation Metric: You loaded the ROUGE metric using the evaluate library.
Calculating and Displaying ROUGE Scores: You used the calculate_metric_on_test_ds function to calculate the ROUGE scores on a small subset (first 10 examples) of the test set and then on the entire test set. You stored these scores in dictionaries and displayed them in pandas DataFrames for easy interpretation.
This entire process, from loading the pre-trained model and data to tokenizing, setting up the training environment, and finally evaluating the fine-tuned model, is the workflow for fine-tuning a sequence-to-sequence model like Pegasus for a specific task, in this case, summarization on the SAMSum dataset.


just give me bullet points without explanation
Okay, here is the workflow you followed in bullet points:

 followed in bullet points:

Setup and Installations
Loading the Model and Tokenizer
Loading and Preparing the Dataset
Tokenizing the Dataset
Setting up for Training
Initializing and Running the Trainer
Evaluating the Model
Loading the Evaluation Metric
Calculating and Displaying ROUGE Scores

In [25]:
## Save the model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [26]:
### Save the tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [27]:
# Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [28]:
from os import pipe
gen_kwargs = {"length_penalty" : 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)
print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)

Device set to use cuda:0


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
